# Usando Optax


* En este notebook nos enfocaremos en Optax. La librería para hacer más fácil la optimización en Jax.

In [41]:
# Importamos librerias
from jax import numpy as jnp
from jax import random, nn, lax
import jax
import optax
import numpy as np

## Cargamos datos

In [4]:
from sklearn.datasets import load_breast_cancer
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

In [5]:
# Cargamos el dataset
X, y = load_breast_cancer(return_X_y=True)
scaler = StandardScaler()
scaler.fit(X)
X_norm = scaler.transform(X)
X_jax = jnp.array(X_norm)
y_jax = jnp.array(y)

In [28]:
y_jax

Array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0,
       1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0,
       1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0,
       0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1,
       1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0,

In [56]:
dataset_size, input_dim = X_jax.shape
BATCH_SIZE = 16
EPOCHS = 5

## Definimos arquitectura

In [57]:
key = random.key(10)
keys = random.split(key, 4) # vamos a usar 4 capas: input, hidden1, hidden2, output

In [58]:
# Definimos los pesos
params = {'input_layer': random.normal(shape=[input_dim, 16], key=keys[0]),
          'hidden_layer1': random.normal(shape=[16, 8], key=keys[1]),
          'hidden_layer2': random.normal(shape=[8, 4], key=keys[2]),
          'output_layer': random.normal(shape=[4, 1], key=keys[3])}

In [59]:
# Definimos la red
def net(x: jnp.array, params: optax.Params) -> jnp.array:
  y = jnp.dot(x, params['input_layer'])
  y = nn.tanh(y)
  y = jnp.dot(y, params['hidden_layer1'])
  y = nn.tanh(y)
  y = jnp.dot(y, params['hidden_layer2'])
  y = nn.tanh(y)
  y = jnp.dot(y, params['output_layer'])
  y = nn.sigmoid(y)

  return y

In [60]:
# Definimos función de pérdida
def loss(params: optax.Params, batch: jnp.array, labels: jnp.array) -> jnp.array:
  y_hat = net(batch, params)
  loss_values = optax.losses.sigmoid_binary_cross_entropy(y_hat, labels).sum(axis=-1)

  return loss_values.mean()

In [61]:
# Definimos función de entrenamiento
def fit(params: optax.Params, data: jnp.array, labels: jnp.array, optimizer: optax.GradientTransformation)->optax.Params:
  # Inicializamos el optimizador (en el caso de Adam por ejemplo, es el momento)
  opt_state = optimizer.init(params)

  # Definimos una función para aplicar en cada paso
  @jax.jit
  def step(params, opt_state, batch, labels):
    # Calculamos gradientes y pérdidas
    loss_value, grads = jax.value_and_grad(loss)(params, batch, labels)

    # Calculamos la actualización a los parámetros (Me regresa el valor que le debo de sumar a los gradientes)
    updates, opt_state = optimizer.update(grads, opt_state, params)

    # Actualizamos parámetros
    params = optax.apply_updates(params, updates)

    return params, opt_state, loss_value

  steps = dataset_size // BATCH_SIZE
  for epoch in range(EPOCHS):
    print(f"Epoch {epoch}")
    for s in range(steps):
      batch = lax.slice_in_dim(data, s*BATCH_SIZE, BATCH_SIZE*(s+1), axis=0)
      batch_labels = lax.slice_in_dim(labels, s*BATCH_SIZE, BATCH_SIZE*(s+1), axis=0)
      params, opt_state, loss_value = step(params, opt_state, batch, batch_labels)
      if s % 1 == 0:
        print(f'step {s}, loss: {loss_value}')

  return params



## Entrenamos modelo

In [62]:
optimizer = optax.adam(learning_rate=1e-2)
trained_params = fit(params, X_jax, y_jax, optimizer)

Epoch 0
step 0, loss: 15.98946475982666
step 1, loss: 14.4599027633667
step 2, loss: 15.38520622253418
step 3, loss: 10.228194236755371
step 4, loss: 11.714997291564941
step 5, loss: 11.666516304016113
step 6, loss: 9.808555603027344
step 7, loss: 11.270360946655273
step 8, loss: 11.449634552001953
step 9, loss: 8.253365516662598
step 10, loss: 11.270712852478027
step 11, loss: 11.272677421569824
step 12, loss: 13.698443412780762
step 13, loss: 11.896186828613281
step 14, loss: 10.743557929992676
step 15, loss: 10.72927474975586
step 16, loss: 13.042450904846191
step 17, loss: 10.753866195678711
step 18, loss: 9.050983428955078
step 19, loss: 7.259209156036377
step 20, loss: 10.737532615661621
step 21, loss: 9.60079288482666
step 22, loss: 9.720134735107422
step 23, loss: 10.764806747436523
step 24, loss: 9.528870582580566
step 25, loss: 8.582788467407227
step 26, loss: 8.344059944152832
step 27, loss: 10.728671073913574
step 28, loss: 9.723578453063965
step 29, loss: 8.444610595703125

In [63]:
# Evaluación
yp_jax = net(X_jax, trained_params)
yp = np.array(yp_jax.tolist())
preds = np.where(yp>0.5, 1, 0)
accuracy_score(y, preds)

0.9314586994727593

# Conclusiones

* Ayuda mucho la librería de optax para no preocuparse por la implementación de la pérdida y de la actualización de gradientes.
* Resalto mucho la importancia de la arquitectura de la red neuronal.
* De los hiperparametros, de los que más afecto fue el batch size, mientras más pequeño parece que funcinó mejor.